In [1]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Cargamos el Set de Entrenamiento y el Test Final

In [2]:
train = pd.read_csv('modelo.csv')
test_final = pd.read_csv('test_final.csv')

id_aviso_postulante = test_final['id']

In [4]:
test_s = 0.25
random_s = 0
n_samples = 100000

no = train['se_postulo'] == 0
si = train['se_postulo'] == 1

train_no = train[no].sample(n_samples)
train_si = train[si].sample(n_samples)

train = pd.merge(train_no, train_si, how='outer')

x = np.array(train[['sexo', 'orden_estudio', 'nivel_laboral', 'tipo_de_trabajo']])
y = np.array(train['se_postulo'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_s, random_state=random_s)

### Valores iniciales para K

In [43]:
print('VALORES INICIALES PARA K')

k_valores = [10,15,20,25,30,35,40,45,50]
mejor_k = 0
mejor_precision = 0

for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = accuracy_score(y_test, pred)
    print('La precisión para k=', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

VALORES INICIALES PARA K
La precisión para k= 10 es:  0.63992
La precisión para k= 15 es:  0.65926
La precisión para k= 20 es:  0.66586
La precisión para k= 25 es:  0.6837
La precisión para k= 30 es:  0.6838
La precisión para k= 35 es:  0.68672
La precisión para k= 40 es:  0.68762
La precisión para k= 45 es:  0.6885
La precisión para k= 50 es:  0.68718
Con k= 45 se obtuvo la mayor precisión: 0.6885


### Valores Grid Search para K

In [44]:
print('VALORES GRID SEARCH PARA K')

k_valores = []

for i in range(mejor_k-2, mejor_k+3):
    k_valores.append(i)
    
for k in k_valores:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train, y_train)
    pred = knn.predict(x_test)
    precision = accuracy_score(y_test, pred)
    print('La precisión para k =', k, 'es:',precision)
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_k = k
        
print('Con k=', mejor_k, 'se obtuvo la mayor precisión:', mejor_precision)

VALORES GRID SEARCH PARA K
La precisión para k = 43 es: 0.68776
La precisión para k = 44 es: 0.68758
La precisión para k = 45 es: 0.6885
La precisión para k = 46 es: 0.68754
La precisión para k = 47 es: 0.6873
Con k= 45 se obtuvo la mayor precisión: 0.6885


## Predecimos el Test Final

En base a los resultados obtenidos con el set de entrenamiento, utilizaremos los hiper-parámetros que consideramos óptimos para predecir las postulaciones del test final.

In [1]:
x_test = np.array(test_final[['sexo', 'orden_estudio', 'tipo_de_trabajo', 'nivel_laboral']])

knn = KNeighborsClassifier(n_neighbors=mejor_k)

# Lo entrenamos con la totalidad del set de datos
knn.fit(x,y)

# Predecimos las postulaciones del set final
pred_final = knn.predict(x_test)

## Submit File

Generamos el Submit File para subir a la competencia de Kaggle

In [ ]:
submit = pd.DataFrame({'id':id_aviso_postulante, 'sepostulo':pred_final})

submit.to_csv('submit_knn.csv', index=False)